In [58]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

table = soup.find('table',class_ = 'wikitable sortable' )
cols = [v.text.replace('\n', '') for v in rows[0].find_all('th')]

rows = table.find_all('tr')
values = []
for v in rows:
    values.append([td.text.strip() for td in v.find_all('td')])
    #print(values)


In [5]:
df = pd.DataFrame(values, columns = cols)

In [6]:
df.shape

(181, 3)

In [7]:
df.isna().sum()

Postal Code     1
Borough         1
Neighborhood    1
dtype: int64

In [8]:
df1 = df.dropna()

In [9]:
to_drop = df1[df1['Borough'] == 'Not assigned'].index
df1.drop(to_drop, axis = 0, inplace = True)
df1.isna().sum()
df1.head()


C:\Users\Mohsin Ali\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postal Code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df1.shape


(103, 3)

# Part 2

In [11]:
coord = pd.read_csv(r'C:\Users\Mohsin Ali\Jupyter\Datasets\IBM Captone\Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
final_df = pd.merge(df1,coord, on = 'Postal Code', how = 'left')

In [13]:
final_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# part 3 - analysis

In [14]:
df3 = final_df[final_df['Borough'] == "Downtown Toronto"].reset_index(drop = True)
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [15]:
df3.drop(['Postal Code'], axis = 1).head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [16]:
#Define Foursquare Credentials and Version
CLIENT_ID = '0MJXHNCGGUCO2KL0YZE1EZVCXIL5NJPLGWFXKZE4T0DDRH2G' # your Foursquare ID
CLIENT_SECRET = 'T1OKFN1XI21O323RFQ3HAQUCJW4WCFDYUBK4NTXFXOLHOLT0' # your Foursquare Secret
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:0MJXHNCGGUCO2KL0YZE1EZVCXIL5NJPLGWFXKZE4T0DDRH2G
CLIENT_SECRET:T1OKFN1XI21O323RFQ3HAQUCJW4WCFDYUBK4NTXFXOLHOLT0


In [17]:
# get the geographical coordinates of Downtown Toronto
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode('Downtown Toronto, ON, Canada')
location.address
toro_lat = location.latitude
toro_long = location.longitude

C:\Users\Mohsin Ali\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
print('Latitude and Longitude for toronoto are : ', toro_lat, toro_long)

Latitude and Longitude for toronoto are :  43.6563221 -79.3809161


In [19]:
import folium

toro_map = folium.Map(location = [toro_lat, toro_long], zoom_start = 11)

#ADDING MARKERS

for lat, lng, lab in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    label = folium.Popup(lab, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 6,
        popup = label,
        color = 'Orange',
        fill = True,
        fill_color = 'Neighborhood',
        fill_opacity = 0.8,
        parse_html = False).add_to(toro_map)
    
toro_map

In [20]:
from folium import plugins
toro_map = folium.Map(location = [toro_lat, toro_long], zoom_start = 11)


cluster = plugins.MarkerCluster().add_to(toro_map)

#ADDING MARKERS

for lat, lng, lab in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    label = folium.Popup(lab, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 6,
        popup = label,
        color = 'Orange',
        fill = True,
        fill_color = 'Neighborhood',
        fill_opacity = 0.8,
        parse_html = False).add_to(cluster)
    
toro_map

In [21]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
downtown_toro_vanues = getNearbyVenues(names = df3['Neighborhood'], latitudes = df3['Latitude'], longitudes = df3['Longitude'], )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [25]:
#Shape of the dataframe
downtown_toro_vanues.shape


(357, 7)

In [26]:
downtown_toro_vanues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,17,17,17,17,17,17
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
"Harbourfront East, Union Station, Toronto Islands",20,20,20,20,20,20


In [27]:
# counting unique vanue category

len(downtown_toro_vanues['Venue Category'].unique())

129

In [28]:
downtown_toro_vanues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [32]:
# one hot encoding

downtown_toro_onehot = pd.get_dummies(downtown_toro_vanues[['Venue Category']], prefix = "", prefix_sep = "")

In [33]:
downtown_toro_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
downtown_toro_onehot['Neighborhood'] = downtown_toro_vanues['Neighborhood']
pd.set_option('display.max_columns', 500)

downtown_toro_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Farmers Market,Fish Market,Flower Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Neighborhood,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toro_grouped = downtown_toro_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toro_grouped.head()


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Farmers Market,Fish Market,Flower Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.05,0.0000,0.05,0.05,0.0,0.0000,0.00,0.05,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.050000,0.0,0.0,0.0,0.00,0.05,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.05,0.00,0.0,0.05,0.05,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.05,0.0,0.00,0.0,0.05,0.0,0.0,0.00,0.00,0.00,0.05,0.0,0.000000,0.0,0.0,0.050000,0.0,0.0,0.00,0.0000,0.0,0.0,0.0,0.00,0.00,0.0000,0.050000,0.0,0.00,0.00,0.0000,0.05,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.0,0.00,0.0,0.0,0.05,0.0,0.0,0.00,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.0625,0.1875,0.125,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0625,0.00,0.00,0.0,0.0625,0.00,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.062500,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0625,0.0,0.0,0.0,0.00,0.00,0.0625,0.000000,0.0,0.00,0.00,0.0625,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,Central Bay Street,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0,0.0,0.05,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0000,0.00,0.00,0.0,0.0000,0.00,0.00,0.05,0.0,0.0,0.000000,0.000000,0.0,0.0,0.05,0.0,0.0,0.0,0.00,0.200000,0.0,0.0,0.0,0.05,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.050000,0.1,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.05,0.05,0.00,0.0,0.000000,0.0,0.0,0.050000,0.0,0.0,0.00,0.0000,0.0,0.0,0.0,0.00,0.05,0.0000,0.000000,0.0,0.00,0.05,0.0000,0.00,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.05,0.0,0.0,0.00,0.0
3,Christie,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0,0.0,0.00,0.0,0.0,0.058824,0.0,0.058824,0.00,0.0000,0.00,0.00,0.0,0.0000,0.00,0.00,0.00,0.0,0.0,0.176471,0.058824,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.058824,0.0,0.0,0.0,

In [55]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toro_grouped['Neighborhood']:
    print(hood)
    temp = downtown_toro_grouped[downtown_toro_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Berczy Park
          venue  freq
0        Museum  0.05
1        Bakery  0.05
2   Coffee Shop  0.05
3  Cocktail Bar  0.05
4    Restaurant  0.05


CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
              venue  freq
0   Airport Service  0.19
1  Airport Terminal  0.12
2           Airport  0.06
3               Bar  0.06
4   Harbor / Marina  0.06


Central Bay Street
                 venue  freq
0          Coffee Shop  0.20
1  Japanese Restaurant  0.10
2     Ramen Restaurant  0.05
3            Gastropub  0.05
4      Bubble Tea Shop  0.05


Christie
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3      Nightclub  0.06
4     Baby Store  0.06


Church and Wellesley
                venue  freq
0         Coffee Shop  0.05
1  Mexican Restaurant  0.05
2        Dance Studio  0.05
3                 Pub  0.05
4    Ramen Restaurant  0.05


Commerce Court, Victoria Hotel
         venue  freq
0 

In [56]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toro_grouped['Neighborhood']

for ind in np.arange(downtown_toro_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toro_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Basketball Stadium,Bakery,Coffee Shop,Museum,Concert Hall,Restaurant,Liquor Store,Breakfast Spot,Seafood Restaurant,Beer Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport,Plane,Boat or Ferry,Coffee Shop,Rental Car Location,Bar,Harbor / Marina,Sculpture Garden
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Modern European Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Comic Shop,Bubble Tea Shop,Sandwich Place
3,Christie,Grocery Store,Café,Park,Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Italian Restaurant,Diner,Candy Store
4,Church and Wellesley,General Entertainment,Dance Studio,Park,Coffee Shop,Pizza Place,Mexican Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop
5,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Bookstore,Bakery,Gastropub,Gym,Gym / Fitness Center,American Restaurant,Restaurant,Museum
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Steakhouse,Pizza Place,General Travel,Pub,Bookstore,Seafood Restaurant,Bakery
7,"Garden District, Ryerson",Café,Tea Room,Comic Shop,Diner,College Rec Center,Sporting Goods Shop,Steakhouse,Ramen Restaurant,Electronics Store,Burrito Place
8,"Harbourfront East, Union Station, Toronto Islands",Park,Plaza,Hotel,Supermarket,Performing Arts Venue,Café,Bubble Tea Shop,Salad Place,Lake,Skating Rink
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Beer Bar,Fish Market,Coffee Shop,Cocktail Bar,Gourmet Shop,Cheese Shop,Caribbean Restaurant,Mexican Restaurant


In [62]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

downtown_toro_grouped_clustering = downtown_toro_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0)
kmeans.fit(downtown_toro_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 1, 0, 1, 1, 1, 0, 1])

In [64]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = df3

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Breakfast Spot,Farmers Market,Chocolate Shop,Pub,Restaurant,Dessert Shop,Performing Arts Venue,Bakery
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Sushi Restaurant,Coffee Shop,Yoga Studio,Mexican Restaurant,Burrito Place,Creperie,Diner,Beer Bar,Discount Store,Smoothie Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Café,Tea Room,Comic Shop,Diner,College Rec Center,Sporting Goods Shop,Steakhouse,Ramen Restaurant,Electronics Store,Burrito Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Gastropub,Coffee Shop,Poke Place,Hotel,Middle Eastern Restaurant,Restaurant,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Basketball Stadium,Bakery,Coffee Shop,Museum,Concert Hall,Restaurant,Liquor Store,Breakfast Spot,Seafood Restaurant,Beer Bar


In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[toro_lat, toro_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters